# 3. Pré-processamento

Nosso modelo só pode prever os jogos com base em dados passados dos times, não a partir das variáveis do jogo a ser previsto (até porque a previsão ocorre antes do jogo). O algoritmo deve se basear apenas nas informações passadas

In [6]:
import pandas as pd
import datetime as dt

In [7]:
df = pd.read_csv('dados_brasileirao_tratados.csv') 
brasileirao = pd.read_csv('jogos_brasileirao.csv')

## Calcular média móvel

Para uma análise de resultado normalmente leva-se em conta os últimos 3 ou 5 jogos disputados pelas equipes. Por conta disso, calcularemos a **média móvel dos últimos 5 jogos de cada time**

### Juntando dataframes

Vou precisar atribuir o ano dos jogos às linhas na base de dados usada para fazer o treinamento. Isso porque terei de agrupar por ano os dados para calcular a média móvel

In [12]:
# Criando uma coluna 'ano' com o ano em que cada jogo aconteceu
try:
    brasileirao['ano'] = pd.to_datetime(brasileirao['data'].str.title(), dayfirst=True).dt.year
    
except AttributeError:
    print('A conversão já foi feita')

In [13]:
brasileirao.rename(columns={'ID': 'partida_id'}, inplace=True)

In [14]:
#Fazendo a junção dos dataframes
df_atualizado = pd.merge(brasileirao[['partida_id', 'ano']], df, on='partida_id')

In [22]:
df_atualizado.head()

,partida_id,ano,rodata,clube,chutes,chutes_no_alvo,posse_de_bola,passes,precisao_passes,faltas,cartao_amarelo,cartao_vermelho,impedimentos,escanteios
0,6126,2018,1,Cruzeiro,12.0,2.0,0.40,425.0,0.81,20.0,3,0,0,7
1,6126,2018,1,Gremio,6.0,2.0,0.60,649.0,0.86,15.0,1,1,3,4
2,6127,2018,1,Vitoria,21.0,6.0,0.66,545.0,0.88,24.0,3,0,2,4
3,6127,2018,1,Flamengo,10.0,5.0,0.34,286.0,0.72,10.0,2,1,2,5
4,6128,2018,1,Santos,22.0,3.0,0.54,532.0,0.90,7.0,0,0,1,4


### Calculando a média móvel

Com a base de dados devidamente ajustada, podemos calcular a média móvel dos últimos 5 jogos de cada time

In [76]:
#Cálculo da média móvel
df_mm = df_atualizado.groupby(['ano', 'clube'])[df_atualizado.iloc[:, 3:].columns].rolling(5).mean()

#Defasagem de variáveis
df_mm = df_mm[df_mm.columns.difference(['ano', 'clube'])].groupby(['ano', 'clube']).shift(1).reset_index()

#Renomeando coluna
df_mm = df_mm.rename(columns = {'level_2':'index'})

#Juntando as duas bases de dado
df_atualizado = df_atualizado.reset_index().iloc[:, :4].merge(df_mm)

## Aproveitamento

Agora precisamos calcular o aproveitamento dos times. Para tal, precisamos dividir o número de pontos obtido pelo número de pontos possíveis de serem conquistados até aquele momento

In [103]:
#Criar variável binária para identificar se um time está jogando em casa ou fora

brasileirao.loc[brasileirao['vencedor'] == brasileirao['mandante'], 'casa'] = 1
brasileirao.loc[brasileirao['vencedor'] != brasileirao['mandante'], 'casa'] = 0

#Iniciou a coluna 'pontos'


In [105]:
brasileirao

,partida_id,rodata,data,hora,mandante,visitante,formacao_mandante,formacao_visitante,tecnico_mandante,tecnico_visitante,vencedor,arena,mandante_Placar,visitante_Placar,mandante_Estado,visitante_Estado,resultado,ano,casa
0,6126,1,14/4/2018,16:00,Cruzeiro,Gremio,4-2-3-1,4-2-3-1,L. Venker de Menezes,Renato Gaúcho,Gremio,Mineirão,0,1,MG,RS,1,2018,0.0
1,6127,1,14/4/2018,19:02,Vitoria,Flamengo,4-2-3-1,4-1-4-1,V. Carmo Mancini,M. Nogueira Barbieri,-,Barradão,2,2,BA,RJ,2,2018,0.0
2,6128,1,14/4/2018,21:03,Santos,Ceara,4-3-3,4-1-4-1,J. Ribeiro Ventura,M. Oliveira Chamusca,Santos,Estádio Municipal Paulo Machado de Carvalho,2,0,SP,CE,0,2018,1.0
3,6129,1,15/4/2018,11:00,America-MG,Sport,4-1-4-1,4-3-2-1,E. Alves Moreira,N. Baptista Júnior,America-MG,Estádio Raimundo Sampaio,3,0,MG,PE,0,2018,1.0
4,6130,1,15/4/2018,16:01,Vasco,Atletico-MG,4-1-4-1,4-1-4-1,J. Mannarino,T. Mendes Larghi,Vasco,Estádio Vasco da Gama,2,1,RJ,MG,0,2018,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1895,8021,38,13/11/2022,16:03,Cuiaba,Coritiba,4-1-4-1,4-1-4-1,A. Cardoso de Oliveira,A. Ferreira,Cuiaba,Arena Pantanal,2,1,MT,PR,0,2022,1.0
1896,8022,38,13/11/2022,16:03,Bragantino,Fluminense,4-2-3-1,4-2-3-1,M. Nogueira Barbieri,F. Diniz Silva,Fluminense,Nabizão,0,1,SP,RJ,1,2022,0.0
1897,8023,38,13/11/2022,16:03,Corinthians,Atletico-MG,4-1-4-1,4-2-3-1,F. J. Monteiro Almeida,A. Stival,Atletico-MG,Neo Química Arena,0,1,SP,MG,1,2022,0.0
1898,8024,38,13/11/2022,16:03,Internacional,Palmeiras,4-2-3-1,4-2-3-1,L. Venker de Menezes,A. Moreira Ferreira,Internacional,Estádio José Pinheiro Borda,3,0,RS,SP,0,2022,1.0
